In [4]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=ae668c786408d9d13648ae5b8643402d5e47d9f37a9bbecf71e4047304843a9a
  Stored in directory: c:\users\тата\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [49]:
!pip install geocoder

In [128]:
!pip install pgeocode
!pip install geopy

In [13]:
!pip install folium

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import geocoder
import pgeocode
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text

In [3]:
soup = BeautifulSoup(data, "html5lib")
table=soup.find('table')

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = ((row.span.text).split('(')[0])
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.shape    

(103, 3)

In [5]:
geolocator = pgeocode.Nominatim('ca')
postal_codes = df['PostalCode'].tolist()
p_code = []
latitudes = []
longitudes = []
for i, postal_code in enumerate(postal_codes):
   
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        p_code.append(postal_code)
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)
        
df_code = pd.DataFrame({'PostalCode':p_code, 'latitudes':latitudes, 'longitudes':longitudes})

In [6]:
df_code['latitudes'] = df_code['latitudes'].fillna(43.6369656)
df_code['longitudes'] = df_code['longitudes'].fillna(-79.615819)

In [7]:
df_merge = df.merge(df_code, how = 'inner')

In [8]:
df_merge

,PostalCode,Borough,Neighborhood,latitudes,longitudes
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
100,M7Y,East Toronto Business,Enclave of M4L,43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


In [9]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [10]:
toronto_map = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat, lon, Borough, Neighborhood, PostalCode in zip(df_merge['latitudes'], 
                                                       df_merge['longitudes'], 
                                                       df_merge['Borough'], 
                                                       df_merge['Neighborhood'],
                                                      df_merge['PostalCode']):
    
    label = '{},{},{}'.format(Borough, Neighborhood, PostalCode)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lon],
    radius = 5,
    popup = label,
    color = 'blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(toronto_map)

toronto_map


In [37]:
CLIENT_ID = 'WJQ13VGLNRME2U5HXXNSX1YOSXDVWJXVU4YU3GF2JBK4ATUY' # your Foursquare ID
CLIENT_SECRET = 'BVSK1HOXJODVQ3UJE2X522S1FBSXBEH2SFHT132FD3M2EVZ3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WJQ13VGLNRME2U5HXXNSX1YOSXDVWJXVU4YU3GF2JBK4ATUY
CLIENT_SECRET:BVSK1HOXJODVQ3UJE2X522S1FBSXBEH2SFHT132FD3M2EVZ3


In [38]:
def getNearbyVenues(Neighborhood, Borough, latitudes, longitudes, radius=500):
    Borough_ = []
    Neighborhood_ = []
    Neighborhood_Latitude = []
    Neighborhood_Longitude = []
    Venue = []
    Venue_Latitude = []
    Venue_Longitude = []
    Venue_Category = []
    
    for neighborhood, borough, lat, lon in zip(Neighborhood, Borough, latitudes, longitudes):
        print(neighborhood)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lon,
        radius,
        LIMIT)
        
        result = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        for v in result:
            Borough_.append(borough)
            Neighborhood_.append(neighborhood),
            Neighborhood_Latitude.append(lat),
            Neighborhood_Longitude.append(lon),
            Venue.append(v['venue']['name']),
            Venue_Latitude.append(v['venue']['location']['lat']),
            Venue_Longitude.append(v['venue']['location']['lng']),
            Venue_Category.append(v['venue']['categories'][0]['name'])
    
    dff = pd.DataFrame({'Borough': Borough_, 'Neighborhood':Neighborhood_, 'Neighborhood_Latitude':Neighborhood_Latitude,
                      'Neighborhood_Longitude':Neighborhood_Longitude, 'Venue':Venue, 'Venue_Latitude':Venue_Latitude,
                      'Venue_Longitude':Venue_Longitude, 'Venue_Category':Venue_Category})
                       
    return(dff)

In [39]:
df_venue = getNearbyVenues(Neighborhood = df_merge['Neighborhood'], Borough = df_merge['Borough'], 
                latitudes = df_merge['latitudes'], longitudes = df_merge['longitudes'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [40]:
toronto_n = pd.get_dummies(df_venue[['Venue_Category']], prefix="", prefix_sep="")
toronto_n['Neighborhood'] = df_venue['Neighborhood']

In [41]:
toronto_n_gr = toronto_n.groupby('Neighborhood').mean().reset_index()
toronto_n_gr

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,Woburn,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,Woodbine Heights,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,York Mills West,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_n_gr['Neighborhood']

for ind in np.arange(toronto_n_gr.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_n_gr.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Badminton Court,Skating Rink,Moving Target,New American Restaurant,Music Venue,Music Store,Museum,Movie Theater
1,"Alderwood, Long Branch",Coffee Shop,Convenience Store,Gym,Pub,Sandwich Place,Pizza Place,Modern European Restaurant,Molecular Gastronomy Restaurant,Mobile Phone Shop,Monument / Landmark
2,"Bathurst Manor, Wilson Heights, Downsview North",Middle Eastern Restaurant,Pizza Place,Deli / Bodega,Fried Chicken Joint,Mediterranean Restaurant,Coffee Shop,Accessories Store,Moving Target,Music Venue,Music Store
3,Bayview Village,Gas Station,Dog Run,Flower Shop,Trail,Park,Music Store,Museum,Moving Target,Movie Theater,Moroccan Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Toy / Game Store,Café,Butcher,Pizza Place,Sushi Restaurant,Boutique,Fast Food Restaurant


In [44]:
kclusters = 5

toronto_n_gr_clustering = toronto_n_gr.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_n_gr_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 




array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_n_gr_merged = df_merge

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_n_gr_merged = toronto_n_gr_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_n_gr_merged.head()


,PostalCode,Borough,Neighborhood,latitudes,longitudes,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,0.0,Food & Drink Shop,Park,Fast Food Restaurant,Accessories Store,Nightclub,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
1,M4A,North York,Victoria Village,43.7276,-79.3148,1.0,Coffee Shop,Pizza Place,Intersection,Portuguese Restaurant,French Restaurant,Hockey Arena,Park,Movie Theater,Music Venue,Music Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,1.0,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Health Food Store,Gym / Fitness Center,Food Truck,Event Space,Electronics Store,Distribution Center
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,1.0,Clothing Store,Coffee Shop,Restaurant,Cosmetics Shop,Women's Store,Men's Store,Food Court,Shoe Store,Toy / Game Store,Bakery
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889,1.0,Sushi Restaurant,Gym,Distribution Center,Bubble Tea Shop,Burrito Place,Coffee Shop,College Theater,Restaurant,Ramen Restaurant,Creperie


In [47]:
toronto_n_gr_merged = toronto_n_gr_merged.dropna().reset_index()

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_n_gr_merged['latitudes'], toronto_n_gr_merged['longitudes'], 
                                  toronto_n_gr_merged['Neighborhood'], toronto_n_gr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1

In [52]:
toronto_n_gr_merged.loc[toronto_n_gr_merged['Cluster Labels'] == 0, toronto_n_gr_merged.columns[[3]+
                                                                list(range(7, toronto_n_gr_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Food & Drink Shop,Park,Fast Food Restaurant,Accessories Store,Nightclub,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
7,Don Mills North,Pool,Furniture / Home Store,Park,Accessories Store,Monument / Landmark,Music Store,Museum,Moving Target,Movie Theater,Moroccan Restaurant
12,Don Mills South,Gym,Trail,River,Park,Accessories Store,Movie Theater,Music Venue,Music Store,Museum,Moving Target
20,Caledonia-Fairbanks,Park,Gym,Beer Store,Sporting Goods Shop,Bakery,Mexican Restaurant,Women's Store,Pharmacy,Music Venue,Middle Eastern Restaurant
26,Hillcrest Village,Park,Residential Building (Apartment / Condo),Accessories Store,New American Restaurant,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Noodle House
31,Scarborough Village,Park,Grocery Store,Spa,Monument / Landmark,Music Store,Museum,Moving Target,Movie Theater,Moroccan Restaurant,Accessories Store
34,The Danforth East,Convenience Store,Intersection,Park,Accessories Store,Movie Theater,New American Restaurant,Music Venue,Music Store,Museum,Moving Target
35,"Harbourfront East, Union Station, Toronto Islands",Café,Park,Music Venue,Harbor / Marina,Music Store,Museum,Moving Target,Movie Theater,Accessories Store,Nightclub
51,"Willowdale, Newtonbrook",Playground,Electronics Store,Park,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater
60,Lawrence Park,Park,Photography Studio,Accessories Store,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Moroccan Restaurant,Monument / Landmark


# Cluster 2

In [53]:
toronto_n_gr_merged.loc[toronto_n_gr_merged['Cluster Labels'] == 1, toronto_n_gr_merged.columns[[3]+
                                                                list(range(7, toronto_n_gr_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Coffee Shop,Pizza Place,Intersection,Portuguese Restaurant,French Restaurant,Hockey Arena,Park,Movie Theater,Music Venue,Music Store
2,"Regent Park, Harbourfront",Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Health Food Store,Gym / Fitness Center,Food Truck,Event Space,Electronics Store,Distribution Center
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Coffee Shop,Restaurant,Cosmetics Shop,Women's Store,Men's Store,Food Court,Shoe Store,Toy / Game Store,Bakery
4,Ontario Provincial Government,Sushi Restaurant,Gym,Distribution Center,Bubble Tea Shop,Burrito Place,Coffee Shop,College Theater,Restaurant,Ramen Restaurant,Creperie
5,Islington Avenue,Pharmacy,Bank,Home Service,Grocery Store,Skating Rink,Park,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Nightclub
...,...,...,...,...,...,...,...,...,...,...,...
93,"First Canadian Place, Underground city",Coffee Shop,Hotel,Café,Restaurant,Gym,Japanese Restaurant,Asian Restaurant,Salad Place,Steakhouse,Deli / Bodega
94,"The Kingsway, Montgomery Road, Old Mill North",Breakfast Spot,Bank,Sushi Restaurant,Burger Joint,Coffee Shop,Bar,Bakery,Smoke Shop,Liquor Store,Restaurant
95,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Mediterranean Restaurant,Bubble Tea Shop,Dance Studio,Fast Food Restaurant,Men's Store
96,Enclave of M4L,Restaurant,Coffee Shop,Yoga Studio,Bank,Department Store,Breakfast Spot,Sushi Restaurant,Bookstore,Deli / Bodega,Italian Restaurant


# Cluster 3

In [54]:
toronto_n_gr_merged.loc[toronto_n_gr_merged['Cluster Labels'] == 2, toronto_n_gr_merged.columns[[3]+
                                                                list(range(7, toronto_n_gr_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,"Wexford, Maryvale",Auto Garage,Accessories Store,Movie Theater,New American Restaurant,Music Venue,Music Store,Museum,Moving Target,Moroccan Restaurant,Noodle House


# Cluster 3

In [55]:
toronto_n_gr_merged.loc[toronto_n_gr_merged['Cluster Labels'] == 3, toronto_n_gr_merged.columns[[3]+
                                                                list(range(7, toronto_n_gr_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,"York Mills, Silver Hills",Pool,Accessories Store,Noodle House,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater
45,Downsview West,Vietnamese Restaurant,Pool,Monument / Landmark,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Moroccan Restaurant,Accessories Store
